In [11]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report
import numpy as np
import tensorflow as tf
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import classification_report
from keras.models import Sequential
from keras.layers import Dense, Embedding, GlobalAveragePooling1D
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.wrappers.scikit_learn import KerasClassifier


In [13]:
data = pd.read_csv("..\dataset\\spam_ham_dataset.csv")

X = data['text']
y = data['label_num']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [14]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)
X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test)

max_seq_length = max([len(seq) for seq in X_train_seq])
X_train_pad = pad_sequences(X_train_seq, maxlen=max_seq_length, padding='post')
X_test_pad = pad_sequences(X_test_seq, maxlen=max_seq_length, padding='post')

In [17]:
model = Sequential()
model.add(Embedding(input_dim=len(tokenizer.word_index)+1, output_dim=64, input_length=max_seq_length))
model.add(GlobalAveragePooling1D())
model.add(Dense(128, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

callback = tf.keras.callbacks.EarlyStopping(
    monitor='loss', 
    patience=3
)
model.fit(X_train_pad, y_train, validation_data=(X_test_pad, y_test), epochs=20, batch_size=16, callbacks=[callback])

Epoch 1/20
259/259 [==============================] - 27s 102ms/step - loss: 0.6062 - accuracy: 0.7084 - val_loss: 0.5915 - val_accuracy: 0.7169
Epoch 2/20
259/259 [==============================] - 28s 108ms/step - loss: 0.5900 - accuracy: 0.7087 - val_loss: 0.5653 - val_accuracy: 0.7169
Epoch 3/20
259/259 [==============================] - 28s 106ms/step - loss: 0.5135 - accuracy: 0.7309 - val_loss: 0.4186 - val_accuracy: 0.7536
Epoch 4/20
259/259 [==============================] - 26s 99ms/step - loss: 0.3493 - accuracy: 0.8530 - val_loss: 0.3034 - val_accuracy: 0.8029
Epoch 5/20
259/259 [==============================] - 26s 100ms/step - loss: 0.2198 - accuracy: 0.9408 - val_loss: 0.1973 - val_accuracy: 0.9063
Epoch 6/20
259/259 [==============================] - 25s 98ms/step - loss: 0.1600 - accuracy: 0.9529 - val_loss: 0.1383 - val_accuracy: 0.9807
Epoch 7/20
259/259 [==============================] - 26s 100ms/step - loss: 0.1158 - accuracy: 0.9756 - val_loss: 0.1568 - val_accu

In [18]:
loss, accuracy = model.evaluate(X_test_pad, y_test)
print(f"loss : {loss}")
print(f"accuracy : {accuracy}")

33/33 [==============================] - 1s 27ms/step - loss: 0.0567 - accuracy: 0.9797
loss : 0.05668860673904419
accuracy : 0.9797101616859436


In [19]:
y_pred_prob = model.predict(X_test_pad)
y_pred = np.round(y_pred_prob).astype(int)
conf_matrix = confusion_matrix(y_test, y_pred)
tn, fp, fn, tp = conf_matrix.ravel()

print("confusion matrix:")
print(conf_matrix)
print("True Positives (TP) :", tp)
print("False Positives (TN) :", tn)

33/33 [==============================] - 4s 27ms/step
Matrice de confusion:
[[724  18]
 [  3 290]]
True Positives (TP) : 290
False Positives (TN) : 724
